# CodeAct: Code-Centric Agent Execution

In [ ]:
import re
import io
from contextlib import redirect_stdout, redirect_stderr
from agentic_patterns.core.agents import get_agent, run_agent

## The CodeAct Pattern

CodeAct treats code execution as the agent's primary action modality. Instead of text-based actions like ReAct, the agent generates actual executable code. The code is run in a sandboxed environment, and the output (including errors) becomes feedback for the next iteration.

## Execution Sandbox

A simple sandbox that executes Python code and captures output. The namespace persists across executions, allowing the agent to build state incrementally.

In [ ]:
class CodeSandbox:
    """Simple Python execution sandbox with persistent namespace."""

    def __init__(self):
        self.namespace = {"__builtins__": __builtins__}

    def execute(self, code: str, timeout: float = 5.0) -> str:
        """Execute code and return combined stdout/stderr output."""
        stdout_capture = io.StringIO()
        stderr_capture = io.StringIO()

        try:
            with redirect_stdout(stdout_capture), redirect_stderr(stderr_capture):
                exec(code, self.namespace)

            output = stdout_capture.getvalue()
            return output if output else "(code executed successfully, no output)"

        except Exception as e:
            return f"Error: {type(e).__name__}: {e}"

## CodeAct Prompt

The prompt instructs the model to think in code. Code blocks are executed, and results are fed back.

In [ ]:
CODEACT_PROMPT = """You are a code-execution agent. Solve tasks by writing and executing Python code.

Rules:
1. Write code in ```python blocks. Each block will be executed and you'll see the output.
2. Use print() to show results - this is how you see what's happening.
3. Variables persist between executions, so you can build on previous code.
4. If you get an error, analyze it and fix your code in the next iteration.
5. When the task is complete, write DONE followed by a brief summary.

Example interaction:
User: Calculate the factorial of 5
Assistant: I'll calculate the factorial of 5.
```python
result = 1
for i in range(1, 6):
    result *= i
print(f"Factorial of 5 is {result}")
```
[Execution output: Factorial of 5 is 120]
DONE: The factorial of 5 is 120.
"""

## The CodeAct Loop

The loop extracts code blocks, executes them, and appends results to the context for the next iteration.

In [ ]:
def extract_code_blocks(text: str) -> list[str]:
    """Extract Python code blocks from markdown-formatted text."""
    pattern = r"```python\s*(.*?)```"
    return re.findall(pattern, text, re.DOTALL)


async def codeact_loop(task: str, max_steps: int = 6) -> str:
    """Run the CodeAct loop until DONE or max steps reached."""
    agent = get_agent(system_prompt=CODEACT_PROMPT)
    sandbox = CodeSandbox()

    trajectory = f"Task: {task}\n"

    for step in range(max_steps):
        agent_run, _ = await run_agent(agent, trajectory)
        response = agent_run.result.output.strip()

        print(f"\n{'=' * 50}")
        print(f"Step {step + 1}")
        print(f"{'=' * 50}")
        print(response)

        trajectory += f"\nAssistant: {response}\n"

        # Extract and execute code blocks first
        code_blocks = extract_code_blocks(response)

        if code_blocks:
            for code in code_blocks:
                code = code.strip()
                if code:
                    print("\n--- Executing ---")
                    output = sandbox.execute(code)
                    print(f"Output: {output}")
                    trajectory += f"\n[Execution output: {output}]\n"

        # Check for completion after executing any code
        if "DONE" in response.upper():
            print(f"\n{'=' * 50}")
            print("Task completed")
            return response

        if not code_blocks:
            trajectory += "\n[No code block found. Please write Python code in ```python blocks.]\n"

    return "Max steps reached."

## Example 1: Data Analysis

The agent analyzes sales data by writing and executing code. Note how it builds state across executions.

In [ ]:
task = """
Analyze this sales data and find:
1. Total revenue
2. Best selling product
3. Average order value

Data:
orders = [
    {"product": "Widget A", "quantity": 5, "price": 10.00},
    {"product": "Widget B", "quantity": 3, "price": 25.00},
    {"product": "Widget A", "quantity": 8, "price": 10.00},
    {"product": "Widget C", "quantity": 2, "price": 50.00},
    {"product": "Widget B", "quantity": 6, "price": 25.00},
]
"""

await codeact_loop(task)

## Example 2: Error Recovery

This example shows how the agent handles and recovers from errors, using them as feedback to fix the code.

In [ ]:
task = """
Calculate the mean and standard deviation of these numbers: [4, 8, 15, 16, 23, 42]
Do not import any libraries - implement the calculations manually.
"""

await codeact_loop(task)